### Update

Now it's fast thanks to [@higepon](https://www.kaggle.com/higepon) and [@romainfabre](https://www.kaggle.com/romainfabre) !


### Description
This is an example of making one dataframe to feed into one model.
The result is [here](https://www.kaggle.com/kokitanisaka/indoorunifiedwifids).

This code is super slow, if you know how to make it faster, please let me know.

The created dataset is here.<br>
https://www.kaggle.com/kokitanisaka/indoorunifiedwifids

The oridinal dataset is here.<br>
https://www.kaggle.com/hiro5299834/indoor-navigation-and-location-wifi-features

In [ ]:
import pandas as pd
import numpy as np
import glob
import os

import multiprocessing
from multiprocessing import Pool

num_cores = multiprocessing.cpu_count()

In [ ]:
ITEMS_TO_TAKE = 100

In [ ]:
base_path = '/kaggle'
feature_dir = f"{base_path}/input/indoor-navigation-and-location-wifi-features"
train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_dir, '*_test.csv')))

In [ ]:
def generate_training_data(train_file):
    train = pd.read_csv(train_file)

    file_name = train_file.split('/')[-1]

    num_of_lines = train.shape[0]
    print(f'{file_name} : {num_of_lines}')

    data = []
    for i in range(num_of_lines):
        if i % 100 == 0:
            print(f'current:{i}')

        tmp = train.iloc[i,1:-4].astype(int).sort_values(ascending=False).head(ITEMS_TO_TAKE)
        target = train.iloc[i,-4:]
        line = [*tmp.index.astype(str), *tmp.values, *target]
        data.append(line)
    data = pd.DataFrame(data)
    data.columns = [f'bssid_{str(i)}' for i in range(ITEMS_TO_TAKE)] + [f'rssi_{str(i)}' for i in range(ITEMS_TO_TAKE)] + ['x','y','floor','path']
    data.to_csv(f'{file_name}', index=False)

In [ ]:
with Pool(num_cores) as pool:
    pool.map(generate_training_data, [t for t in train_files])  

In [ ]:
def generate_test_data(test_file):
    train = pd.read_csv(test_file)

    file_name = test_file.split('/')[-1]

    num_of_lines = train.shape[0]
    print(f'{file_name} : {num_of_lines}')

    data = []
    for i in range(num_of_lines):
        if i % 100 == 0:
            print(f'current:{i}')

        tmp = train.iloc[i,1:-4].astype(int).sort_values(ascending=False).head(ITEMS_TO_TAKE)
        target = train.iloc[i, [-1]]

        line = [*tmp.index.astype(str), *tmp.values, *target]
        data.append(line)
    data = pd.DataFrame(data)
    data.columns = [f'bssid_{str(i)}' for i in range(ITEMS_TO_TAKE)] + [f'rssi_{str(i)}' for i in range(ITEMS_TO_TAKE)] + ['site_path_timestamp']
    data.to_csv(f'{file_name}', index=False)

In [ ]:
with Pool(num_cores) as pool:
    pool.map(generate_test_data, [t for t in test_files])  